In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [66]:
all_master = pd.read_csv('data/Master.csv')
all_salaries = pd.read_csv('data/Salaries.csv')
all_fielding = pd.read_csv('data/Fielding.csv')
all_batting = pd.read_csv('data/Batting.csv')
batting = all_batting[['playerID', 'yearID', 'teamID', 'H', 'BB', 'HBP', 'AB', 'SF']]
batting = batting[batting['SF'] > 0]
batting['OBP'] = (batting['H'] + batting['BB'] + batting['HBP'])/(batting['AB'] + batting['BB'] + batting['HBP'] + batting['SF'])
master = all_master[['playerID', 'nameFirst', 'nameLast']]
fielding = all_fielding[['playerID', 'yearID', 'POS', 'G']]
salaries = all_salaries[['playerID', 'yearID', 'salary']]
merged = pd.merge(pd.merge(pd.merge(master, batting), fielding), salaries)
merged.groupby(['playerID', 'yearID'])
merged

,playerID,nameFirst,nameLast,yearID,teamID,H,BB,HBP,AB,SF,OBP,POS,G,salary
0,abbotje01,Jeff,Abbott,1998,CHA,68,9,0,244,5,0.298450,CF,38,175000
1,abbotje01,Jeff,Abbott,1998,CHA,68,9,0,244,5,0.298450,DH,2,175000
2,abbotje01,Jeff,Abbott,1998,CHA,68,9,0,244,5,0.298450,LF,20,175000
3,abbotje01,Jeff,Abbott,1998,CHA,68,9,0,244,5,0.298450,OF,76,175000
4,abbotje01,Jeff,Abbott,1998,CHA,68,9,0,244,5,0.298450,RF,27,175000
5,abbotje01,Jeff,Abbott,1999,CHA,9,5,0,57,1,0.222222,LF,17,255000
6,abbotje01,Jeff,Abbott,1999,CHA,9,5,0,57,1,0.222222,OF,17,255000
7,abbotje01,Jeff,Abbott,2000,CHA,59,21,2,215,1,0.343096,CF,33,255000
8,abbotje01,Jeff,Abbott,2000,CHA,59,21,2,215,1,0.343096,LF,20,255000
9,abbotje01,Jeff,Abbott,2000,CHA,59,21,2,215,1,0.343096,OF,65,255000


In [61]:
print(grouped)

OBP = (Hits + Bases on Balls(walks) + Hit By Pitch)/(At Bats + Bases on Balls(walks) + Hit By Pitch + Sacrifice Flies)